In [22]:
import numpy as np

In [23]:
file_path = "./ddi_with_type_latest.txt"
with open(file_path, "r") as f:
    readfile = f.read()

In [57]:
reflection_list = []
vector_set = set()
max_vector = 0
for item in readfile.split(sep="\n"):
    if len(item.split(sep="\t")) == 3:
        single_item = item.split(sep="\t")
        reflection_list.append(single_item)
        vector_set.add(single_item[1])
        vector_set.add(single_item[0])
        max_vector = max(max_vector, max(int(single_item[0]), int(single_item[1])))

## Q1 Find the number of nodes and edges in the network

In [69]:
# number of nodes
len(vector_set)

1603

In [70]:
# number of edges
len(reflection_list)

38405

## Q2 Find nodes with the top 20 highest degree and the avarage of degree of all nodes

In [61]:
# Use dict to restore the degree of every nodes
node_degree = {str(i):0 for i in range(len(vector_set))}
for reflect in reflection_list:
    node_degree[reflect[0]] += int(reflect[2])
    node_degree[reflect[1]] += int(reflect[2])
node_degree

{'0': 1054,
 '1': 1110,
 '2': 292,
 '3': 952,
 '4': 49,
 '5': 7546,
 '6': 1377,
 '7': 83,
 '8': 221,
 '9': 1166,
 '10': 75,
 '11': 235,
 '12': 497,
 '13': 1158,
 '14': 895,
 '15': 165,
 '16': 492,
 '17': 12,
 '18': 550,
 '19': 4180,
 '20': 186,
 '21': 3927,
 '22': 7990,
 '23': 3827,
 '24': 4851,
 '25': 2999,
 '26': 2692,
 '27': 2448,
 '28': 5469,
 '29': 3101,
 '30': 1779,
 '31': 2420,
 '32': 3006,
 '33': 7598,
 '34': 2398,
 '35': 2902,
 '36': 5306,
 '37': 6026,
 '38': 6852,
 '39': 5676,
 '40': 4002,
 '41': 850,
 '42': 7074,
 '43': 2620,
 '44': 2463,
 '45': 5374,
 '46': 3395,
 '47': 6984,
 '48': 1538,
 '49': 3084,
 '50': 1589,
 '51': 2930,
 '52': 2939,
 '53': 5542,
 '54': 6712,
 '55': 2618,
 '56': 1661,
 '57': 842,
 '58': 1092,
 '59': 7412,
 '60': 924,
 '61': 1520,
 '62': 2925,
 '63': 8978,
 '64': 1944,
 '65': 7946,
 '66': 2711,
 '67': 2255,
 '68': 3663,
 '69': 6101,
 '70': 1200,
 '71': 964,
 '72': 240,
 '73': 49,
 '74': 2432,
 '75': 3726,
 '76': 969,
 '77': 2419,
 '78': 2154,
 '79': 64

In [67]:
# Order nodes by sum_degree and cut the top20
a = sorted(node_degree.items(), key=lambda x: x[1], reverse=True)
a[:20]

[('1440', 11010),
 ('1339', 10887),
 ('848', 10458),
 ('1358', 10293),
 ('441', 10021),
 ('1079', 9912),
 ('1411', 9624),
 ('578', 9463),
 ('118', 9259),
 ('347', 9259),
 ('63', 8978),
 ('611', 8977),
 ('1518', 8938),
 ('446', 8806),
 ('419', 8620),
 ('742', 8597),
 ('362', 8570),
 ('1261', 8465),
 ('1418', 8385),
 ('944', 8247)]

In [74]:
# Calculate the average degree of all nodes
average_degree = sum(node_degree.values())//len(vector_set)
average_degree

2388